In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
mem=pd.read_csv("/home/piai/바탕화면/2. Big Data 분석/members_completed.csv")
mem.head()

,회원번호,회원상태,성별,나이,등록카드,결혼,구독여부,주소지,세부주소지
0,18764160,정상회원,여,68,농협중앙회,기혼,False,서울특별시,성동구
1,18792000,정상회원,남,83,연결앱결제,기혼,False,강원도,강릉시
2,18942336,정상회원,여,39,신한은행,기혼,False,인천광역시,중구
3,18949760,정상회원,여,73,기업은행,기혼,NaN,강원도,홍천군
4,19391488,정상회원,여,52,연결앱결제,기혼,False,대전광역시,중구


In [3]:
prd=pd.read_csv("/home/piai/바탕화면/2. Big Data 분석/products_completed.csv")
mem.head()

,회원번호,회원상태,성별,나이,등록카드,결혼,구독여부,주소지,세부주소지
0,18764160,정상회원,여,68,농협중앙회,기혼,False,서울특별시,성동구
1,18792000,정상회원,남,83,연결앱결제,기혼,False,강원도,강릉시
2,18942336,정상회원,여,39,신한은행,기혼,False,인천광역시,중구
3,18949760,정상회원,여,73,기업은행,기혼,NaN,강원도,홍천군
4,19391488,정상회원,여,52,연결앱결제,기혼,False,대전광역시,중구


In [4]:
sales=pd.read_csv("/home/piai/바탕화면/2. Big Data 분석/Sales.csv")
sales.head()

,회원번호,회원상태,구매수량,구매금액,주문일시,배송시작일,배송완료일,사용 적립금,사용 포인트 네이버,주문취소여부,주문시간,제품번호,월,일,요일
0,1032097472,정상회원,1.0,7083,2021-01-02,NaN,NaN,0,0,1,13:00:00,100021783V2_1337,1,2,Saturday
1,1032097472,정상회원,1.0,29865,2021-01-02,2021-01-02,2021-01-02,0,0,0,13:00:00,100022137V2_1606,1,2,Saturday
2,1032097472,정상회원,1.0,23164,2021-01-02,2021-01-02,2021-01-03,0,0,0,13:00:00,100021452V2_1113,1,2,Saturday
3,369152832,정상회원,1.0,16655,2021-01-02,2021-01-02,2021-01-03,0,0,0,13:00:00,10002931V2_708,1,2,Saturday
4,1032097472,정상회원,1.0,8423,2021-01-02,NaN,NaN,0,0,1,13:00:00,100022085V2_1559,1,2,Saturday


In [6]:
mem['연령대'] = np.where(mem['나이']<20, '10대', np.where(mem['나이']<30, '20대',np.where(mem['나이']<40,'30대',np.where(mem['나이']<50,'40대',np.where(mem['나이']<60,'50대',np.where(mem['나이']<101, '60대이상',"eroror"))))))

In [7]:
sales['배송시작일'] = pd.to_datetime(sales['배송시작일'])
sales['배송완료일'] = pd.to_datetime(sales['배송완료일'])
sales['주문일시'] = pd.to_datetime(sales['주문일시'])

In [8]:
sales['배송기간']=sales['배송완료일'] - sales['배송시작일']

In [9]:
sales['주문_도착']=sales['배송완료일'] - sales['주문일시']

In [10]:
sales['주문_배송']=sales['배송시작일'] - sales['주문일시']

In [11]:
sales['배송기간'].value_counts()

0 days    294009
1 days    241832
2 days     96403
Name: 배송기간, dtype: int64

In [12]:
sales['주문_도착'].value_counts()


1 days    240778
0 days    230052
2 days    130126
3 days     27207
4 days      4081
Name: 주문_도착, dtype: int64

In [13]:
df=pd.merge(sales,prd ,on="제품번호", how="left")

In [14]:
df=pd.merge(df,mem ,on="회원번호", how="left")

In [15]:
df['물품분류'].value_counts()

채소          157491
가공식품/간편식    116519
고기/계란        98037
간식           67067
해산물          45944
음료/주류        32386
과일           30052
곡물           23343
유제품          23231
소스류          19425
가루/기름        18919
생활용품         17580
화장품           6759
건강식품          4998
식기/편백          944
Name: 물품분류, dtype: int64

In [19]:
def sinsunfood (x):
    if (x == "고기/계란") or (x == "과일") or (x == "유제품") or (x == "해산물") or (x == "채소"):
        return '신선식품'
    elif (x=='생활용품') or (x=="식기/편백") or (x=="화장품"): 
        return '식품아님'
    else:
        return '신선식품아님'
df['신선식품'] = df['물품분류'].apply(sinsunfood)

In [21]:
tt = df.groupby(['연령대','구독여부'],as_index=False)['신선식품'].value_counts()

In [27]:
tt

,연령대,구독여부,신선식품,count
0,10대,False,신선식품,192
1,10대,False,신선식품아님,190
2,10대,False,식품아님,27
3,10대,True,신선식품아님,15
4,10대,True,식품아님,2
5,20대,False,신선식품아님,20288
6,20대,False,신선식품,19162
7,20대,False,식품아님,1660
8,20대,True,신선식품아님,3250
9,20대,True,신선식품,3163


In [30]:
tt.pivot(index=["연령대", "구독여부"], columns=["신선식품"],values=["count"])

count                  
신선식품           식품아님     신선식품   신선식품아님
연령대   구독여부                           
10대   False    27.0    192.0    190.0
      True      2.0      NaN     15.0
20대   False  1660.0  19162.0  20288.0
      True    279.0   3163.0   3250.0
30대   False  6725.0  79564.0  71242.0
      True   1321.0  15762.0  14598.0
40대   False  5524.0  73505.0  54819.0
      True   1018.0  15440.0  10946.0
50대   False  2193.0  40636.0  27994.0
      True    580.0  10123.0   6926.0
60대이상 False  1197.0  27202.0  18023.0
      True    203.0   6230.0   3884.0

-> 구독 소비자와 미구독 소비자간 구매 패턴이 비슷함에도 불구하고 미구독 소비자가 많음
구독 유도를 위한 추가 분석 필요(원인 분석 Ex. 구독 서비스 마케팅 실패, 구독료 부담, 구독 혜택에 대한 충분한 설명 부족 등등)

In [23]:
tt[tt['구독여부'] == True]

,연령대,구독여부,신선식품,count
3,10대,True,신선식품아님,15
4,10대,True,식품아님,2
8,20대,True,신선식품아님,3250
9,20대,True,신선식품,3163
10,20대,True,식품아님,279
14,30대,True,신선식품,15762
15,30대,True,신선식품아님,14598
16,30대,True,식품아님,1321
20,40대,True,신선식품,15440
21,40대,True,신선식품아님,10946


In [24]:
tt[tt['구독여부'] == False]

,연령대,구독여부,신선식품,count
0,10대,False,신선식품,192
1,10대,False,신선식품아님,190
2,10대,False,식품아님,27
5,20대,False,신선식품아님,20288
6,20대,False,신선식품,19162
7,20대,False,식품아님,1660
11,30대,False,신선식품,79564
12,30대,False,신선식품아님,71242
13,30대,False,식품아님,6725
17,40대,False,신선식품,73505
